In [ ]:


%%time
import s3fs
import zarr

url = "s3://hrrrzarr/sfc/20170101/20170101_00z_anl.zarr"
fs = s3fs.S3FileSystem(anon=True)
store = zarr.open(s3fs.S3Map(url, s3=fs))



CPU times: user 474 ms, sys: 68.2 ms, total: 542 ms
Wall time: 1.2 s


In [ ]:


%%time
thousand_mb_tmp = store["1000mb/TMP/1000mb/TMP"]
surface_tmp = store["surface/TMP/surface/TMP"]
hcdc = store["high_cloud_layer/HCDC/high_cloud_layer/HCDC"]
rh = store["2m_above_ground/RH/2m_above_ground/RH"]
spfh = store["2m_above_ground/SPFH/2m_above_ground/SPFH"]
dpt = store["2m_above_ground/DPT/2m_above_ground/DPT"]
pot = store["2m_above_ground/POT/2m_above_ground/POT"]
tmp = store["2m_above_ground/TMP/2m_above_ground/TMP"]



CPU times: user 62 ms, sys: 2.1 ms, total: 64.1 ms
Wall time: 613 ms


In [3]:
!pip install cartopy
!pip install s3fs
!pip install zarr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 54.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 37.7 MB/s eta 0:00:00
  Created wheel for asciitree: filename=asciitree-0.3.3-

In [27]:

import xarray as xr
import zarr
import s3fs

import cartopy as ccrs
fs = s3fs.S3FileSystem(anon=True)
chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
projection = ccrs.crs.LambertConformal(central_longitude=-97.5,
                             central_latitude=38.5,
                             standard_parallels=[38.5])

def get_nearest_point(projection, chunk_index, longitude, latitude):
    x, y = projection.transform_point(longitude, latitude, ccrs.crs.PlateCarree())
    return chunk_index.sel(x=x, y=y, method="nearest")

nearest_point = get_nearest_point(projection, chunk_index, -111.8910, 40.7608)

first_index = int(nearest_point.index_y.values)
second_index = int(nearest_point.index_x.values)


In [ ]:
thisdict = {
  "TIME": "TMP",
}
arr = pd.DataFrame.from_dict(thisdict,orient='index')

In [ ]:
%%time
import datetime
import numpy as np
import pandas as pd

# thisdict = {
#   "TIME": "TMP",
# }

start = datetime.datetime.strptime("20211214_15", '%Y%m%d_%H')
# forecast_length = 67000
tmp_analysis = []
i = 0

# data = np.empty(forecast_length, dtype=[('Analysis Time', 'datetime64[s]'), ('TMP', 'float')])

for time_delta in range(forecast_length):
    # construct the aws path
    analysis_time = start + datetime.timedelta(hours=time_delta)
    # print(analysis_time)


    string_date = analysis_time.strftime('%Y%m%d')
    string_hour   = analysis_time.strftime('%H')

    url = 's3://hrrrzarr/sfc/%s/%s_%sz_anl.zarr' %(string_date, string_date, string_hour)

    try:
        store = zarr.open(s3fs.S3Map(url, s3=fs))
        print(f"Successfully opened {url}")


    # val = store["surface/TMP/surface/TMP"][first_index, second_index]






        thisdict[analysis_time] = store["surface/TMP/surface/TMP"][first_index, second_index]


    # append the data we need
        tmp_analysis.append(store["surface/TMP/surface/TMP"][first_index, second_index])
        # Do your processing here
    except Exception as e:
        print(f"Failed to open {url}: {e}")
        # Continue with the next iteration if an error occurs
        continue





In [ ]:
%%time
import datetime
import s3fs
import zarr
import numpy as np
import pandas as pd



start = datetime.datetime.strptime("20160828_00", '%Y%m%d_%H')
forecast_length = 67000
tmp_analysis = []
i = 0

#data = np.empty(forecast_length, dtype=[('Analysis Time', 'datetime64[s]'), ('TMP', 'float')])

for time_delta in range(forecast_length):
    # construct the aws path
    analysis_time = start + datetime.timedelta(hours=time_delta)
    print(analysis_time)


    string_date = analysis_time.strftime('%Y%m%d')
    string_hour   = analysis_time.strftime('%H')
    url = 's3://hrrrzarr/sfc/%s/%s_%sz_anl.zarr' %(string_date, string_date, string_hour)

    # open the zarr group
    store = zarr.open(s3fs.S3Map(url, s3=fs))

    # val = store["surface/TMP/surface/TMP"][first_index, second_index]





    i = i+1
    thisdict[analysis_time] = store["surface/TMP/surface/TMP"][first_index, second_index]


    # append the data we need
    tmp_analysis.append(store["surface/TMP/surface/TMP"][first_index, second_index])

In [29]:
import pandas as pd
arr = pd.DataFrame()



In [31]:
display(arr)

""


In [8]:


%%time
import datetime
import numpy as np
import pandas as pd
import s3fs
import zarr

# thisdict = {
#   "TIME": "TMP",
# }

start = datetime.datetime.strptime("20170512_20", '%Y%m%d_%H')
forecast_length = 36960
tmp_analysis = []
i = 0

# data = np.empty(forecast_length, dtype=[('Analysis Time', 'datetime64[s]'), ('TMP', 'float')])

for time_delta in range(forecast_length):
    # construct the aws path
    analysis_time = start + datetime.timedelta(hours=time_delta)
    # print(analysis_time)


    string_date = analysis_time.strftime('%Y%m%d')
    string_hour   = analysis_time.strftime('%H')

    url = 's3://hrrrzarr/sfc/%s/%s_%sz_anl.zarr' %(string_date, string_date, string_hour)
    # url = 's3://hrrrzarr/sfc/%s/%s_%sz_anl.zarr' %(string_date, string_date, string_hour)

    try:
        store = zarr.open(s3fs.S3Map(url, s3=fs))
        print(f"Successfully opened {url}")


    # val = store["surface/TMP/surface/TMP"][first_index, second_index]
        pres_val = store["surface/PRES/surface/PRES"][first_index, second_index]
        #APCP_fcst = store["surface/APCP_acc_fcst/surface/APCP_acc_fcst"][first_index, second_index]
        cape = store["surface/CAPE/surface/CAPE"][first_index, second_index]
        cin = store["surface/CIN/surface/CIN"][first_index, second_index]
        cnwat = store["surface/CNWAT/surface/CNWAT"][first_index, second_index]
        dlwrf = store["surface/DLWRF/surface/DLWRF"][first_index, second_index]
        dswrf = store["surface/DSWRF/surface/DSWRF"][first_index, second_index]
        fricv = store["surface/FRICV/surface/FRICV"][first_index, second_index]
        gflux = store["surface/GFLUX/surface/GFLUX"][first_index, second_index]
        gust = store["surface/GUST/surface/GUST"][first_index, second_index]
        hgt = store["surface/HGT/surface/HGT"][first_index, second_index]
        hpbl = store["surface/HPBL/surface/HPBL"][first_index, second_index]
        icec = store["surface/ICEC/surface/ICEC"][first_index, second_index]
        land = store["surface/LAND/surface/LAND"][first_index, second_index]
        lhtfl = store["surface/LHTFL/surface/LHTFL"][first_index, second_index]
        sfcr = store["surface/SFCR/surface/SFCR"][first_index, second_index]
        shtfl = store["surface/SHTFL/surface/SHTFL"][first_index, second_index]
        snod = store["surface/SNOD/surface/SNOD"][first_index, second_index]
        snowc = store["surface/SNOWC/surface/SNOWC"][first_index, second_index]
        ulwrf = store["surface/ULWRF/surface/ULWRF"][first_index, second_index]
        uswrf = store["surface/USWRF/surface/USWRF"][first_index, second_index]
        vbdsf = store["surface/VBDSF/surface/VBDSF"][first_index, second_index]
        vddsf = store["surface/VDDSF/surface/VDDSF"][first_index, second_index]
        vgtyp = store["surface/VGTYP/surface/VGTYP"][first_index, second_index]
        vis = store["surface/VIS/surface/VIS"][first_index, second_index]
        weasd= store["surface/WEASD/surface/WEASD"][first_index, second_index]







        # thisdict[analysis_time] = store["surface/TMP/surface/TMP"][first_index, second_index]
        arr.at[analysis_time,"Surface Pressure"] = pres_val
        arr.at[analysis_time,"Convective Available Potential Energy"] = cape
        arr.at[analysis_time,"Convective Inhibition"] = cin
        arr.at[analysis_time,"Plant Canopy Surface Water"] = cnwat
        arr.at[analysis_time,"Downward Long-Wave Rad. Flux"] = dlwrf
        arr.at[analysis_time,"Downward Short-Wave Rad. Flux"] = dswrf
        arr.at[analysis_time,"Frictional Velocity"] = fricv
        arr.at[analysis_time,"Ground Heat Flux"] = gflux
        arr.at[analysis_time,"Wind Gust "] = gust
        arr.at[analysis_time,"Height of the Level of Free Convection"] =  hgt
        arr.at[analysis_time,"Planetary Boundary Layer Height"] = hpbl
        arr.at[analysis_time,"Ice Cover"] = icec
        arr.at[analysis_time,"Land Cover (0=sea, 1=land)"] = land
        arr.at[analysis_time,"Latent Heat Net Flux"] = lhtfl
        arr.at[analysis_time,"Surface Roughness "] = sfcr
        arr.at[analysis_time,"Sensible Heat Net Flux"] = shtfl
        arr.at[analysis_time,"Snow Depth"] = snod
        arr.at[analysis_time,"Snow Cover"] = snowc
        arr.at[analysis_time,"Upward Long-Wave Rad. Flux "] = ulwrf
        arr.at[analysis_time,"Upward Short-Wave Rad. Flux"] = uswrf
        arr.at[analysis_time,"Visible Beam Downward Solar Flux "] = vbdsf
        arr.at[analysis_time,"Visible Diffuse Downward Solar Flux"] = vddsf
        arr.at[analysis_time,"Vegetation Type"] = vgtyp
        arr.at[analysis_time,"Visibility "] = vis
        arr.at[analysis_time,"Water Equivalent of Accumulated Snow Depth"] = weasd





    # append the data we need
        tmp_analysis.append(store["surface/TMP/surface/TMP"][first_index, second_index])
        # Do your processing here
    except Exception as e:
        print(f"Failed to open {url}: {e}")
        # Continue with the next iteration if an error occurs
        continue





Failed to open s3://hrrrzarr/sfc/20170512/20170512_20z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170512/20170512_21z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170512/20170512_22z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170512/20170512_23z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170513/20170513_00z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170513/20170513_01z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170513/20170513_02z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170513/20170513_03z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170513/20170513_04z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170513/20170513_05z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170513/20170513_06z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170513/20170513_07z_anl.zarr: '.zgroup'
Failed to open s3://hrrrzarr/sfc/20170513/20170513_08z_anl.zarr: '.zgroup'
Failed to open s3://hrrrz

KeyboardInterrupt: 

In [33]:
%%time
import datetime
import numpy as np
import pandas as pd
import s3fs
import zarr

# thisdict = {
#   "TIME": "TMP",
# }

start = datetime.datetime.strptime("20230721_00", '%Y%m%d_%H')
forecast_length = 3936
tmp_analysis = []
i = 0

# data = np.empty(forecast_length, dtype=[('Analysis Time', 'datetime64[s]'), ('TMP', 'float')])

for time_delta in range(forecast_length):
    # construct the aws path
    analysis_time = start + datetime.timedelta(hours=time_delta)
    # print(analysis_time)


    string_date = analysis_time.strftime('%Y%m%d')
    string_hour   = analysis_time.strftime('%H')

    url = 's3://hrrrzarr/sfc/%s/%s_%sz_anl.zarr' %(string_date, string_date, string_hour)
    # url = 's3://hrrrzarr/sfc/%s/%s_%sz_anl.zarr' %(string_date, string_date, string_hour)

    try:
        store = zarr.open(s3fs.S3Map(url, s3=fs))
        print(f"Successfully opened {url}")


        temp = store["surface/TMP/surface/TMP"][first_index, second_index]
        smoke = store["8m_above_ground/MASSDEN/8m_above_ground/MASSDEN"][first_index, second_index]
        Vertically_integrated_smoke = store["entire_atmosphere_single_layer/COLMD/entire_atmosphere_single_layer/COLMD"][first_index, second_index]
        dew_point = store["2m_above_ground/DPT/2m_above_ground/DPT"][first_index, second_index]
       	wind_speed = store["10m_above_ground/WIND_max_fcst/10m_above_ground/WIND_max_fcst"][first_index, second_index]
        relative_humidity = store["2m_above_ground/RH/2m_above_ground/RH"][first_index, second_index]
        specific_humidity = store["2m_above_ground/SPFH/2m_above_ground/SPFH"][first_index, second_index]
        # pres_val = store["surface/PRES/surface/PRES"][first_index, second_index]
        # #APCP_fcst = store["surface/APCP_acc_fcst/surface/APCP_acc_fcst"][first_index, second_index]
        # cape = store["surface/CAPE/surface/CAPE"][first_index, second_index]
        # cin = store["surface/CIN/surface/CIN"][first_index, second_index]
        # cnwat = store["surface/CNWAT/surface/CNWAT"][first_index, second_index]
        dlwrf = store["surface/DLWRF/surface/DLWRF"][first_index, second_index]
        dswrf = store["surface/DSWRF/surface/DSWRF"][first_index, second_index]
        # fricv = store["surface/FRICV/surface/FRICV"][first_index, second_index]
        gflux = store["surface/GFLUX/surface/GFLUX"][first_index, second_index]
        gust = store["surface/GUST/surface/GUST"][first_index, second_index]
        hgt = store["surface/HGT/surface/HGT"][first_index, second_index]
        # hpbl = store["surface/HPBL/surface/HPBL"][first_index, second_index]
        # icec = store["surface/ICEC/surface/ICEC"][first_index, second_index]
        # land = store["surface/LAND/surface/LAND"][first_index, second_index]
        # lhtfl = store["surface/LHTFL/surface/LHTFL"][first_index, second_index]
        # sfcr = store["surface/SFCR/surface/SFCR"][first_index, second_index]
        # shtfl = store["surface/SHTFL/surface/SHTFL"][first_index, second_index]
        # snod = store["surface/SNOD/surface/SNOD"][first_index, second_index]
        # snowc = store["surface/SNOWC/surface/SNOWC"][first_index, second_index]
        ulwrf = store["surface/ULWRF/surface/ULWRF"][first_index, second_index]
        uswrf = store["surface/USWRF/surface/USWRF"][first_index, second_index]
        # vbdsf = store["surface/VBDSF/surface/VBDSF"][first_index, second_index]
        # vddsf = store["surface/VDDSF/surface/VDDSF"][first_index, second_index]
        # vgtyp = store["surface/VGTYP/surface/VGTYP"][first_index, second_index]
        # vis = store["surface/VIS/surface/VIS"][first_index, second_index]
        # weasd= store["surface/WEASD/surface/WEASD"][first_index, second_index]







        arr.at[analysis_time,"Temprature"] = temp
        arr.at[analysis_time,"Smoke_8m_above_ground"] = smoke
        arr.at[analysis_time,"Vertically Integrated Smoke"] = Vertically_integrated_smoke
        arr.at[analysis_time,"Dew Point"] = dew_point
        arr.at[analysis_time,"Wind Speed"] = wind_speed
        arr.at[analysis_time,"Relative Humidity"] = relative_humidity
        arr.at[analysis_time,"Specific Humidity"] = specific_humidity
        #arr.at[analysis_time,"Surface Pressure"] = pres_val
        #arr.at[analysis_time,"Convective Available Potential Energy"] = cape
        # arr.at[analysis_time,"Convective Inhibition"] = cin
        # arr.at[analysis_time,"Plant Canopy Surface Water"] = cnwat
        arr.at[analysis_time,"Downward Long-Wave Rad. Flux"] = dlwrf
        arr.at[analysis_time,"Downward Short-Wave Rad. Flux"] = dswrf
        # arr.at[analysis_time,"Frictional Velocity"] = fricv
        arr.at[analysis_time,"Ground Heat Flux"] = gflux
        arr.at[analysis_time,"Wind Gust "] = gust
        arr.at[analysis_time,"Height of the Level of Free Convection"] =  hgt
        # arr.at[analysis_time,"Planetary Boundary Layer Height"] = hpbl
        # arr.at[analysis_time,"Ice Cover"] = icec
        # arr.at[analysis_time,"Land Cover (0=sea, 1=land)"] = land
        # arr.at[analysis_time,"Latent Heat Net Flux"] = lhtfl
        # arr.at[analysis_time,"Surface Roughness "] = sfcr
        # arr.at[analysis_time,"Sensible Heat Net Flux"] = shtfl
        # arr.at[analysis_time,"Snow Depth"] = snod
        # arr.at[analysis_time,"Snow Cover"] = snowc
        arr.at[analysis_time,"Upward Long-Wave Rad. Flux "] = ulwrf
        arr.at[analysis_time,"Upward Short-Wave Rad. Flux"] = uswrf
        # arr.at[analysis_time,"Visible Beam Downward Solar Flux "] = vbdsf
        # arr.at[analysis_time,"Visible Diffuse Downward Solar Flux"] = vddsf
        # arr.at[analysis_time,"Vegetation Type"] = vgtyp
        # arr.at[analysis_time,"Visibility "] = vis
        # arr.at[analysis_time,"Water Equivalent of Accumulated Snow Depth"] = weasd





    # append the data we need
        tmp_analysis.append(store["surface/TMP/surface/TMP"][first_index, second_index])
        # Do your processing here
    except Exception as e:
        print(f"Failed to open {url}: {e}")
        # Continue with the next iteration if an error occurs
        continue


Successfully opened s3://hrrrzarr/sfc/20230721/20230721_00z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_01z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_02z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_03z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_04z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_05z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_06z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_07z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_08z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_09z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_10z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_11z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_12z_anl.zarr
Successfully opened s3://hrrrzarr/sfc/20230721/20230721_13z_anl.zarr
Successfully opened s3://hrrrzarr/

In [19]:
# from google.colab import drive
# drive.mount('/content/drive')

arr.to_csv('20240214_19_to_20240924_21_rq.csv')


SyntaxError: invalid syntax (<ipython-input-10-46811c8ebf62>, line 1)

In [37]:
import pandas as pd

df1 = pd.read_csv('Weather data/20201202_12_to_20210829_23_rq.csv', index_col = 0)
df2 = pd.read_csv('Weather data/20210830_00_to_20230413_50_rq.csv', index_col = 0)
df3 = pd.read_csv('Weather data/20230413_06_to_20230926_15_rq.csv', index_col = 0)
df4 = pd.read_csv('Weather data/20230701_00_to_20230720_23_rq.csv', index_col = 0)
df5 = pd.read_csv('Weather data/20240101_00_20240214_18_rq.csv', index_col = 0)
df6 = pd.read_csv('Weather data/20240214_19_to_20240924_21_rq.csv', index_col = 0)




dff = pd.concat([df1,df2,df3,df4,arr,df5,df6])

dff.to_csv('data_from_20201202_12_with_smoke.csv')
